In [1]:
import numpy as np
from scipy.special import gammaincc  # Função igamc

def approximate_entropy_test(sequence: str, m: int):
    """
    Executa o Approximate Entropy Test (Entropia Aproximada) conforme NIST 2.13.

    Parâmetros:
    - sequence (str): sequência binária (ex: "110010...")
    - m (int): tamanho dos blocos (m e m+1 serão testados)

    Retorna:
    - ApEn (float): entropia aproximada
    - chi_squared (float): estatística do teste
    - p_value (float): valor-p
    """
    n = len(sequence)

    # Função para contar frequências de todos os blocos possíveis de tamanho m
    def get_block_frequencies(seq, block_size):
        padded_seq = seq + seq[:block_size - 1]  # Etapa 1: extensão cíclica
        counts = {}
        for i in range(n):
            block = padded_seq[i:i + block_size]
            counts[block] = counts.get(block, 0) + 1
        return counts

    # Função para calcular phi
    def calculate_phi(freq_counts):
        total = sum(freq_counts.values())
        phi = 0.0
        for count in freq_counts.values():
            p = count / total
            phi += p * np.log(p)
        return phi

    # Etapas 2-5: frequências e cálculo de φ(m) e φ(m+1)
    freq_m = get_block_frequencies(sequence, m)
    freq_m1 = get_block_frequencies(sequence, m + 1)

    phi_m = calculate_phi(freq_m)
    phi_m1 = calculate_phi(freq_m1)

    # Etapa 6: cálculo da entropia aproximada, estatística χ² e p-valor
    ApEn = phi_m - phi_m1
    chi_squared = 2.0 * n * (np.log(2) - ApEn)
    p_value = gammaincc(2 ** (m - 1), chi_squared / 2.0)

    return ApEn, chi_squared, p_value


# ===============================
# 🔍 EXEMPLO NIST (Seção 2.13.8)
# ===============================
if __name__ == "__main__":
    sequence = (
        "11001001000011111101101010100010001000010110100011"
        "00001000110100110001001100011001100010100010111000"
    )
    m = 2

    ApEn, chi2, p_val = approximate_entropy_test(sequence, m)

    print("==== Approximate Entropy Test ====")
    print(f"Sequência: {sequence[:50]}...")  # Mostra parte da sequência
    print(f"Tamanho da sequência (n): {len(sequence)}")
    print(f"Tamanho do bloco (m): {m}")
    print(f"ApEn(m): {ApEn:.6f}")
    print(f"Chi² (observado): {chi2:.6f}")
    print(f"P-valor: {p_val:.6f}")
    print("Conclusão:", "✅ RANDOM" if p_val >= 0.01 else "❌ NÃO ALEATÓRIA")


==== Approximate Entropy Test ====
Sequência: 11001001000011111101101010100010001000010110100011...
Tamanho da sequência (n): 100
Tamanho do bloco (m): 2
ApEn(m): 0.665393
Chi² (observado): 5.550792
P-valor: 0.235301
Conclusão: ✅ RANDOM
